# Prep steps:
## To incorporate new data:
1. **Log in to [REDCap Q1K Project](https://tacc-redcap.bic.mni.mcgill.ca/redcap_v13.1.27/index.php?pid=78&__record_cache_complete=1)**
2. **Export reports** (*last done Mar 17, 2025*)
    1. Click name of report from lefthand side:
         - #16 - MR acquisition + feedback
         - #20 - MR Demographics
    2. Click button "Export Data"
    3. Choose export format:
         - MR acquisition + feedback (export as **LABELS**)
         - MR Demographics (export as **RAW DATA**)
    4. Click "Export Data"
    5. Click Excel CSV icon to download
    6. Click "Close"
3. **Move reports from Downloads folder to this script's directory**:
   `/export02/data/heatherh/Q1K/scripts/`
4. **Update list of BIC DICOM directories**
   - In terminal, run `/export02/data/heatherh/Q1K/scripts/get_DICOM_dirs.sh`
   
## To align participant IDs (only need to do once):
4. **Create Record ID lookup table**       (*last done Jan 20, 2025*)
    - Copy and paste "Record ID" column from REDCap "Record Status Dashboard" into a text file
    - Delete "Record ID" heading
   

---
# Run code:
## SETUP
Set paths, read in csv/txt files

In [68]:
# Import modules
import os
import pandas as pd
from pathlib import Path
import numpy as np
import glob
import shutil
import csv
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
#!pip install natsort #run this if natsort is not installed
from natsort import natsorted
import statistics
print('All packages ready to go!')

All packages ready to go!


In [69]:
# Import all relevant .csv/.txt files
scriptPath='/export02/data/heatherh/Q1K/scripts/'
os.chdir(scriptPath)
scriptDir=os.getcwd()
files=os.listdir(scriptDir)

reportAcq='*Q1KDatabase-MRAcquisition*'
reportDem='*Q1KDatabase-MRDemographics_DATA_2*'
subListName='*REDCap_RecordID_CustomLabels.txt' 
subListBIC='Q1K_dicom_dirs.txt'

filePathsAcq=glob.glob(scriptDir+'/'+reportAcq) 
filePathsDem=glob.glob(scriptDir+'/'+reportDem) 
filePathsSub=glob.glob(scriptDir+'/'+subListName) 
filePathBIC=glob.glob(scriptDir+'/'+subListBIC)[0] 

# If multiple downloads, choose the most recent (sorted in reverse chronological by default)
if len(filePathsAcq)>1:
    filePathAcq=filePathsAcq[-1]
else: 
    filePathAcq=filePathsAcq[0]
if len(filePathsDem)>1:
    filePathDem=filePathsDem[-1]
else: 
    filePathDem=filePathsDem[0]
if len(filePathsSub)>1:
    filePathSub=filePathsSub[-1]
else: 
    filePathSub=filePathsSub[0]

print('MR Acquisition File:',filePathAcq)
print('MR Demographics File:',filePathDem)
print('Sub Lookup Table:',filePathSub)
print('BIC Sub List:',filePathBIC)

MR Acquisition File: /export02/data/heatherh/Q1K/scripts/Q1KDatabase-MRAcquisitionFeedbac_DATA_LABELS_2025-03-17_2017.csv
MR Demographics File: /export02/data/heatherh/Q1K/scripts/Q1KDatabase-MRDemographics_DATA_2025-03-17_2020.csv
Sub Lookup Table: /export02/data/heatherh/Q1K/scripts/REDCap_RecordID_CustomLabels.txt
BIC Sub List: /export02/data/heatherh/Q1K/scripts/Q1K_dicom_dirs.txt


## ORGANIZE MR INFO
Format MR acquisitions, demographics, etc. into tables

In [70]:
# Bring report data into Python array
# Acquisition Data
data_temp=[] 
with open(filePathAcq,newline='') as reportFileAcq:
    rawDataAcq=csv.reader(reportFileAcq,delimiter=',')
    for row in rawDataAcq:
        data_temp.append(row)
dataAcq=np.array(data_temp[0])  
#print('Headers:',dataAcq)
for row in data_temp[1:]:
    dataAcq=np.vstack([dataAcq,row])
print('Raw dataAcq size:',dataAcq.shape)

#Demographic Data
data_temp=[] 
with open(filePathDem,newline='') as reportFileDem:
    rawDataDem=csv.reader(reportFileDem,delimiter=',')
    for row in rawDataDem:
        data_temp.append(row)
dataDem=np.array(data_temp[0])    
#print('Headers:',dataDem)
for row in data_temp[1:]:
    dataDem=np.vstack([dataDem,row])
print('Raw dataDem size:',dataDem.shape)

Raw dataAcq size: (94, 32)
Raw dataDem size: (2505, 11)


In [83]:
# Clean up data of blank/unwanted rows
rowsToRemoveAcq=[]
for iRow in range(0,len(dataAcq)):
    testCols=dataAcq[iRow,2:]
    if (not any(testCols)) or ('Irini' in dataAcq[iRow,0]) or ('SCORING' in dataAcq[iRow,0]) or ('TEST' in dataAcq[iRow,0]): #Outputs True if any data exist, or False if all blank
        rowsToRemoveAcq.append(iRow)
dataAcqClean=np.delete(dataAcq,rowsToRemoveAcq,axis=0)
print('Clean dataAcq size:',dataAcqClean.shape)
#print(dataAcqClean[:,0])
#print(pd.DataFrame(dataAcqClean))

rowsToRemoveDem=[]
for iRow in range(0,len(dataDem)):
    testCols=dataDem[iRow,2:]
    if (not any(testCols)) or ('Irini' in dataDem[iRow,0]) or ('SCORING' in dataDem[iRow,0]) or ('TEST' in dataDem[iRow,0]): #Outputs True if any data exist, or False if all blank
        rowsToRemoveDem.append(iRow)
dataDemClean=np.delete(dataDem,rowsToRemoveDem,axis=0)
print('Clean dataDem size:',dataDemClean.shape)
print(pd.DataFrame(dataDemClean))

Clean dataAcq size: (93, 32)
Clean dataDem size: (983, 11)
            0                       1             2             3   \
0    record_id       redcap_event_name  mri_birthday  enr2_pro_dob   
1            2            intake_arm_3    1978-08-02                 
2            2         mri_visit_arm_3                               
3            2  retest_mri_visit_arm_3                               
4            4            intake_arm_3    2012-08-17                 
..         ...                     ...           ...           ...   
978   530-3010            intake_arm_1                  2010-08-16   
979   530-3011            intake_arm_1                  1975-11-06   
980   530-3012            intake_arm_1                  1979-01-04   
981   530-3013            intake_arm_1                  2008-06-18   
982   530-3014            intake_arm_1                  1977-09-02   

              4             5              6              7   \
0    enrlmnt_sex  enr2_pro_sex  eeg_

In [72]:
# Read in Record ID Lookup Table
data_temp=[] 
with open(filePathSub,newline='') as subLookupFile:
    rawDataSub=csv.reader(subLookupFile,delimiter=',')
    for row in rawDataSub:
        row_temp=row[0] #allows string to be split
        row_new=row_temp.split(' ')
        IDparts_temp=row_new[2]
        IDparts=IDparts_temp.split('_')
        if len(IDparts) > 1: # a few records don't have IDs (so show as '' [len=1])
            data_temp.append([row_new[0],row_new[2],IDparts[1],IDparts[2][-4:],IDparts[3]])

subLookup=pd.DataFrame(data_temp)
print(subLookup)

            0                     1    2     3   4
0          21       Q1K_MHC_20021_P  MHC  0021   P
1          40       Q1K_HSJ_10040_P  HSJ  0040   P
2          41      Q1K_HSJ_10040_M1  HSJ  0040  M1
3          42       Q1K_MHC_20042_P  MHC  0042   P
4          43       Q1K_HSJ_10043_P  HSJ  0043   P
..        ...                   ...  ...   ...  ..
462  529-4024   Q1K_OIM_4529-4024_P  OIM  4024   P
463  529-4025  Q1K_OIM_4529-4024_M1  OIM  4024  M1
464  529-4026   Q1K_OIM_4529-4026_P  OIM  4026   P
465  529-4027  Q1K_OIM_4529-4026_M1  OIM  4026  M1
466  529-4028  Q1K_OIM_4529-4026_F1  OIM  4026  F1

[467 rows x 5 columns]


In [73]:
# Extract which subs have MR data
MRsubIDs=[]
for iRow in range(1,len(dataDemClean)):
    MRdate=dataDemClean[iRow,9]
    if any(MRdate): #Outputs True if any data exist, or False if all blank
        MRsubID=dataDemClean[iRow,0]
        # Make all Record IDs 3 digits long for sorting by padding with zeros if necessary
        #if len(MRsubID) < 3:
        #    MRsubID = MRsubID.zfill(3)
        #else:
        #    MRsubID = MRsubID[-4:]    
        MRsubIDs.append(MRsubID)
#print('Record IDs:',MRsubIDs)
print('Number of MR scans:',len(MRsubIDs))
MRsubs_temp=np.unique(MRsubIDs)
#print('Number of unique MR scans (Q1K + pilot):',len(MRsubs_temp))

# Compare MR subs to Q1K IDs [translated from chatgpt)
MRsubIdx = []
for MRsub in MRsubs_temp:
    subIdx = subLookup.index[subLookup.iloc[:, 0] == MRsub].tolist()
    if subIdx:  # If subIdx is not empty
        MRsubIdx.extend(subIdx)

MRsubsInfo_temp = pd.concat([subLookup.iloc[[0], :], subLookup.loc[MRsubIdx, :]]) # Extract rows based on MRsubIdx
MRsubsInfo = MRsubsInfo_temp.iloc[1:, :].values #remove header/row labels
MRsubsInfo = np.array(natsorted(MRsubsInfo)) #sorts by record ID incorporating numeric magnitude (e.g., 21 < 104)
#print(MRsubsInfo)
#MRsubsInfo = MRsubsInfo[MRsubsInfo[:,3].argsort()] #sort by subID [note: better to sort by record ID]
MRsubs = MRsubsInfo[:, 0] # Extract first column without header

print('Number of Q1K subs with MR:',len(MRsubs))
print('Record IDs of Q1K subs with MR:',MRsubs)
#print('MR sub info:',MRsubsInfo)

Number of MR scans: 97
Number of Q1K subs with MR: 68
Record IDs of Q1K subs with MR: ['21' '42' '43' '45' '62' '65' '66' '78' '83' '84' '87' '88' '89' '104'
 '106' '107' '120' '121' '122' '128' '146' '152' '153' '171' '174' '175'
 '177' '178' '179' '180' '181' '182' '204' '205' '219' '222' '223' '231'
 '232' '233' '234' '244' '256' '257' '265' '298' '525-1003' '525-1009'
 '525-1010' '525-1011' '525-1028' '525-1029' '525-1030' '525-1031'
 '525-1045' '525-1046' '525-1047' '525-1048' '525-1052' '525-1054'
 '525-1055' '525-1068' '525-1069' '525-1073' '525-1083' '525-1109'
 '525-1110' '525-1111']


In [74]:
# Piece together demographics for MR subs
colRecordID=0
colBirth1=2
colBirth2=3
colSex1=4
colSex2=5
colDiag=6
colGen=7
colMRdate=9

dataAnalysis=np.array(['RecordID','SubID','Age','Sex','Diag_behav','Diag_gene','Gene_copy','MR_date'])

for iSub in MRsubs:
    #print(iSub)
    subRows=dataDemClean[(dataDemClean[:,colRecordID] == iSub)] #find rows with data for that sub 

    Q1Kidx=(MRsubsInfo[:,colRecordID] == iSub)
    Q1Kid=MRsubsInfo[Q1Kidx,1][0]
    #print(Q1Kid)
    
    subBirth=[] #False
    subSex=[] #False
    subDiag_full=[] #False
    subGen=[] #False
    subMRdate=[]
    for row in subRows:
        #birth
        if not subBirth and any(row[colBirth1]):
            subBirth=row[colBirth1]
        elif not subBirth and any(row[colBirth2]):
            subBirth=row[colBirth2]
      
        #sex
        if not subSex and any(row[colSex1]):
            subSex=row[colSex1]
        elif not subSex and any(row[colSex2]):
            subSex=row[colSex2]

        #diagnosis
        if not subDiag_full and any(row[colDiag]):
            subDiag_full=row[colDiag]
        
        #genetics
        if not subGen and any(row[colGen]):
            subGen=row[colGen]
        
        #MR info
        if not subMRdate and any(row[colMRdate]):
            subMRdate=row[colMRdate]
   
    subAge_long=relativedelta(datetime.strptime(subMRdate,"%Y-%m-%d"),datetime.strptime(subBirth,"%Y-%m-%d"))
    subAge=round(subAge_long.years +subAge_long.months/12 + subAge_long.days/365.25,2)

    if not subSex: #some are blank?
        subSex='NA'
    if not subDiag_full:
        subDiag_full='NA'
    if not subGen:
        subGen='NA'

    subData=np.array([iSub,Q1Kid,subAge,subSex,subDiag_full,subDiag_full,subGen,subMRdate])
    dataAnalysis=np.vstack([dataAnalysis,subData])

dataAnalysis_df=pd.DataFrame(dataAnalysis)
dataAnalysis_df.columns=dataAnalysis_df.iloc[0] #turn first row into Header
dataAnalysis_df=dataAnalysis_df[1:]
print(dataAnalysis_df)

0   RecordID                 SubID    Age Sex Diag_behav Diag_gene Gene_copy  \
1         21       Q1K_MHC_20021_P  44.42   2         NA        NA        NA   
2         42       Q1K_MHC_20042_P  30.97  NA          2         2        NA   
3         43       Q1K_HSJ_10043_P  17.08   2        2,3       2,3         1   
4         45      Q1K_HSJ_10043_F1  58.13   2          3         3         1   
5         62       Q1K_HSJ_10062_P  40.77   1          3         3         1   
..       ...                   ...    ...  ..        ...       ...       ...   
64  525-1073   Q1K_HSJ_1525-1073_P  18.72   1        2,3       2,3         1   
65  525-1083   Q1K_HSJ_1525-1083_P  15.15   2          2         2        NA   
66  525-1109   Q1K_HSJ_1525-1109_P  14.01   2          2         2        NA   
67  525-1110  Q1K_HSJ_1525-1109_M1  46.62   1          2         2        NA   
68  525-1111  Q1K_HSJ_1525-1109_F1  48.01   2          1         1        NA   

0      MR_date  
1   2024-10-01  
2   2

In [75]:
# Decode/reformat responses 
allSubCats=['SubCategory']
allSubIDs=['SubNumber']
allSubSexes=['Sex']
allSubDiag_beh=['Diagnosis_behav']
allSubDiag_gen=['Diagnosis_gene']
allSubGene=['Genotype']

for row in dataAnalysis:
    subSex=row[3]
    subDiag=row[4]
    subGene=row[6]

    # Sex
    if "1" in subSex:
        allSubSexes.append('female')
    elif "2" in subSex:
        allSubSexes.append('male')
    elif "NA" in subSex:
        allSubSexes.append(subSex) #NA

    # Diagnosis
    subDiag_behav="unknown"
    subDiag_gene="unknown"
    if "4" in subDiag:
        subDiag_behav='unknown_TBD' 
        subDiag_gene='unknown_TBD'
    if "1" in subDiag: # or "NA" in subDiag:
        subDiag_behav='unaffected' 
        subDiag_gene='non-carrier'
    if "2" in subDiag:
        subDiag_behav='affected'
    if "3" in subDiag:
        subDiag_gene='carrier'    
    if "NA" in subDiag:
        subDiag_behav='NA' 
        subDiag_gene='NA'
    
    if "Diag" not in subDiag: #only add if not first row
        allSubDiag_beh.append(subDiag_behav)
        allSubDiag_gen.append(subDiag_gene)

    # Genetics
    subGenetype="unknown"
    if "1" in subGene:
        subGenetype='deletion'
    if "2" in subGene:
        subGenetype='duplication'
    if "Gene" not in subGene: #only add if not first row
        allSubGene.append(subGenetype)

dataDecoded=np.vstack([dataAnalysis[:,0],np.hstack([allSubIDs,MRsubsInfo[:,3]]),np.hstack([allSubCats,MRsubsInfo[:,4]]),dataAnalysis[:,2],np.array(allSubSexes),np.array(allSubDiag_beh),np.array(allSubDiag_gen),np.array(allSubGene)]).T
dataDecoded_df=pd.DataFrame(dataDecoded)
dataDecoded_df.columns=dataDecoded_df.iloc[0]
dataDecoded_df=dataDecoded_df[1:]
print(dataDecoded_df)

0   RecordID SubNumber SubCategory    Age     Sex Diagnosis_behav  \
1         21      0021           P  44.42    male              NA   
2         42      0042           P  30.97      NA        affected   
3         43      0043           P  17.08    male        affected   
4         45      0043          F1  58.13    male         unknown   
5         62      0062           P  40.77  female         unknown   
..       ...       ...         ...    ...     ...             ...   
64  525-1073      1073           P  18.72  female        affected   
65  525-1083      1083           P  15.15    male        affected   
66  525-1109      1109           P  14.01    male        affected   
67  525-1110      1109          M1  46.62  female        affected   
68  525-1111      1109          F1  48.01    male      unaffected   

0  Diagnosis_gene  Genotype  
1              NA   unknown  
2         unknown   unknown  
3         carrier  deletion  
4         carrier  deletion  
5         carrier  de

In [76]:
# Add scan acquisition notes
scanCols = list(range(6,17)) #where "Done" status for each scan is listed
scanCols.append(29) #where the whole scan is marked as "complete"

scanNames = dataAcqClean[0,scanCols]
#print(scanNames)
scansDecoded=np.hstack([dataDecoded,np.empty([len(dataDecoded),len(scanCols)])])

for iSub in range(0,len(MRsubs)+1):
    
    if iSub == 0:
        scansDecoded[iSub,-(len(scanCols)):]=scanNames #header
    else:
        dataRow = np.where(dataAcqClean[:, 0] == MRsubs[iSub-1])[0] 
        #print(MRsubs[iSub-1],': ', dataRow)
        if dataRow: #if not empty (Record ID 265?)
            scansDecoded[iSub,-(len(scanCols)):]=dataAcqClean[dataRow,scanCols]

scansDecoded_df=pd.DataFrame(scansDecoded)
scansDecoded_df.columns=scansDecoded_df.iloc[0]
scansDecoded_df=scansDecoded_df[1:]
print(scansDecoded_df)

0   RecordID SubNumber SubCategory    Age     Sex Diagnosis_behav  \
1         21      0021           P  44.42    male              NA   
2         42      0042           P  30.97      NA        affected   
3         43      0043           P  17.08    male        affected   
4         45      0043          F1  58.13    male         unknown   
5         62      0062           P  40.77  female         unknown   
..       ...       ...         ...    ...     ...             ...   
64  525-1073      1073           P  18.72  female        affected   
65  525-1083      1083           P  15.15    male        affected   
66  525-1109      1109           P  14.01    male        affected   
67  525-1110      1109          M1  46.62  female        affected   
68  525-1111      1109          F1  48.01    male      unaffected   

0  Diagnosis_gene  Genotype b1_map(1) b1_map(2)           t1w_mprage  \
1              NA   unknown      Done      Done                 Done   
2         unknown   unknown

In [77]:
#Read in list of BIC Dicom Files
data_temp=[] 
with open(filePathBIC,newline='') as BIClookupFile:
    rawDataBIC=csv.reader(BIClookupFile,delimiter=',')
    for row in rawDataBIC:   
        data_temp.append(row)
        
dataBIC_temp=np.array(data_temp) 
dataBIC=dataBIC_temp[dataBIC_temp[:,1].argsort()] #sort by subID
dataBIC_df=pd.DataFrame(dataBIC)
print(dataBIC_df)

            0     1   2                                                  3
0       20021  0021   P  /data_/q1k/data//id20021/P/Q1K_MHC_20021_P_202...
1       20042  0042   P  /data_/q1k/data//id20042/P/Q1K_MCH_20042_P_202...
2       10043  0043   P  /data_/q1k/data//id10043/P/Q1K_HSJ_10043_P_202...
3       10043  0043  F1  /data_/q1k/data//id10043/F1/Q1K_HSJ_10043_F1_2...
4       10062  0062   P                  /data_/q1k/data//id10062/P/*Q1K*/
..        ...   ...  ..                                                ...
64  1525_1073  1073   P  /data_/q1k/data//id1525_1073/P/Q1K_HSJ_1525_10...
65  1525_1083  1083   P  /data_/q1k/data//id1525_1083/P/Q1K_HSJ_1525_10...
66  1525_1109  1109   P  /data_/q1k/data//id1525_1109/P/Q1K_HSJ_1525_11...
67  1525_1109  1109  M1  /data_/q1k/data//id1525_1109/M1/Q1K_HSJ_1525_1...
68  1525_1109  1109  F1  /data_/q1k/data//id1525_1109/F1/Q1K_HSJ_1525_1...

[69 rows x 4 columns]


In [78]:
#Compare REDCap subs with BIC Dicom Files
scansDecoded_BIC=np.hstack([scansDecoded,np.empty([len(scansDecoded),1])])

for iSub in range(0,len(dataBIC)):
    
    if iSub == 0:
        scansDecoded_BIC[iSub,-1]='BIC_DICOM_dir' #header
    else:
        subID = dataBIC[iSub-1,1]
        subCat = dataBIC[iSub-1,2]
        subDicomDir = dataBIC[iSub-1,3]

        # Add Dicom dir to corresponding row [translated from chatgpt)
        subIdx1 = np.where(scansDecoded_BIC[:, 1] == subID)  
        subIdx2 = np.where(scansDecoded_BIC[:, 2] == subCat) 
        dataRow = np.intersect1d(subIdx1,subIdx2)
        scansDecoded_BIC[dataRow,-1]=subDicomDir
        
# Check for missing scans -- NOTE: FIX THIS?
missingSubs=[]
for iSub in range(1,len(scansDecoded_BIC)):
    subRow = scansDecoded_BIC[iSub,:]

    # Add known errors/incomplete scans
    if '0255' in subRow[1] and 'M1' in subRow[2]:
        scansDecoded_BIC[iSub,-1]='No MR data collected' # participant pulled out right after AAscout
    elif '0265' in subRow[1] and 'P' in subRow[2]:
        scansDecoded_BIC[iSub,-1]='No MR data collected' # participant did not fit head coil     

    if not scansDecoded_BIC[iSub,-1]:
        missingSubs.append(scansDecoded_BIC[iSub,1]+scansDecoded_BIC[iSub,2])
        #print('REDCap subs without BIC scans: ', scansDecoded_BIC[iSub,1])
        
scansDecoded_BIC_df=pd.DataFrame(scansDecoded_BIC)
scansDecoded_BIC_df.columns=scansDecoded_BIC_df.iloc[0]
scansDecoded_BIC_df=scansDecoded_BIC_df[1:]
print(scansDecoded_BIC_df)



0   RecordID SubNumber SubCategory    Age     Sex Diagnosis_behav  \
1         21      0021           P  44.42    male              NA   
2         42      0042           P  30.97      NA        affected   
3         43      0043           P  17.08    male        affected   
4         45      0043          F1  58.13    male         unknown   
5         62      0062           P  40.77  female         unknown   
..       ...       ...         ...    ...     ...             ...   
64  525-1073      1073           P  18.72  female        affected   
65  525-1083      1083           P  15.15    male        affected   
66  525-1109      1109           P  14.01    male        affected   
67  525-1110      1109          M1  46.62  female        affected   
68  525-1111      1109          F1  48.01    male      unaffected   

0  Diagnosis_gene  Genotype b1_map(1) b1_map(2)  ... func-resting func-cloudy  \
1              NA   unknown      Done      Done  ...         Done        Done   
2        

In [79]:
# Output missing scans, if any
if missingSubs:
    print('REDCap subs without BIC scans: ', missingSubs)
else:
    print('All REDCap subs have BIC scans accounted for!')

REDCap subs without BIC scans:  ['1109F1']


## RUN DESCRIPTIVE STATS
Aggregate number of usable scans, participant demographics, etc.

In [84]:
# Run stats

doneRows = list(range(8, 19))  
removeIncompleteSubs = []
for iSub in range(1, len(scansDecoded_BIC)):  
    testCols = scansDecoded_BIC[iSub, doneRows]
    #print(iSub,' testCols:',testCols)
    if (testCols == 'Done').sum() > 2:
        continue
    else:
        removeIncompleteSubs.append(iSub)
incompleteIDs= [sub-1 for sub in removeIncompleteSubs] #since MRsubsInfo index starts at 0
print('Subs without usable scan data:',MRsubsInfo[incompleteIDs,3]+MRsubsInfo[incompleteIDs,4])

# Remove incomplete subs
attemptedData = np.delete(scansDecoded_BIC,removeIncompleteSubs,axis=0)
print('Total # of subs with partial data:',len(attemptedData)-1) #subtract header row
#print(pd.DataFrame(attemptedData))

removePartialSubs = []
for iSub in range(1, len(attemptedData)):  
    testCols = attemptedData[iSub, doneRows]
    if any(testCols == 'Not done/incomplete') or any(testCols == ''):
        removePartialSubs.append(iSub)
partialIDs= [sub-1 for sub in removePartialSubs] #since MRsubsInfo index starts at 0
print('Subs with partial scan data:',MRsubsInfo[partialIDs,3]+MRsubsInfo[partialIDs,4])

# Remove partial subs        
completedData = np.delete(attemptedData,removePartialSubs,axis=0)
print('Total # of subs with complete data:',len(completedData)-1) #subtract header row
#print(pd.DataFrame(completedData))

Subs without usable scan data: ['0255F1' '0255M1' '0265P']
Total # of subs with partial data: 65
Subs with partial scan data: ['0064M1' '0181P' '0231S1']
Total # of subs with complete data: 62


In [81]:
# Count how many subs completed each scan
scanHeaders = attemptedData[0,doneRows]
scanData = pd.DataFrame(columns=['Total']) #(columns=scanHeaders)

for iScan in range(0,len(scanHeaders)):
    n=0
    for iSub in range(1,len(attemptedData)):
        if 'Not done/incomplete' in attemptedData[iSub,doneRows[iScan]] or not attemptedData[iSub,doneRows[iScan]]:
            continue
        else:
            n+=1
    scanData.loc[scanHeaders[iScan]]=n
    
print('Scan counts (Total N =',len(attemptedData)-1,'):')    
print(scanData)

Scan counts (Total N = 65 ):
              Total
b1_map(1)        65
b1_map(2)        65
t1w_mprage       65
func-resting     64
func-cloudy      64
fmap-AP          64
fmap-PA          64
dwi_38dir_AP     65
dwi_70dir_AP     63
dwi_b0_PA        63
t1w_mp2rage      61


In [82]:
# Aggregate demographics
demogHeaders=['Count','Age_Mean','Age_Range','Sex_Female','Sex_Male','Sex_Other','NDD_Affected','NDD_NotAffected','NDD_Unknown']
demogRows=['P','M1','F1','S*','Total']
demogData=pd.DataFrame(columns=demogHeaders)

catCol=attemptedData[1:,2]
ageCol=attemptedData[1:,3]
sexCol=attemptedData[1:,4]
NDDcol=attemptedData[1:,5]

for iRow in demogRows:

    if 'Total' in iRow:
        rowIdx = list(range(0,len(attemptedData)-1))
    elif 'S' in iRow:
        rowIdx1 = np.where(catCol == 'S1')[0]
        rowIdx2 = np.where(catCol == 'S2')[0]
        rowIdx3 = np.where(catCol == 'S3')[0]
        rowIdx4 = np.where(catCol == 'S4')[0]
        rowIdx = np.concatenate([rowIdx1,rowIdx2,rowIdx3,rowIdx4])
    else:
        rowIdx = np.where(catCol == iRow)[0]   
    
    #Total    
    catTotal=len(rowIdx)
    demogData.loc[iRow,'Count'] = catTotal

    #Age
    currAges_temp=ageCol[rowIdx]
    currAges=[float(age) for age in currAges_temp]
    ages_mean=round(statistics.mean(currAges),2)
    ages_min=min(currAges)
    ages_max=max(currAges)

    demogData.loc[iRow,'Age_Mean'] = ages_mean
    demogData.loc[iRow,'Age_Range'] = [ages_min, ages_max]

    #Sex
    currSexes=sexCol[rowIdx]
    count_fem=0
    count_mal=0
    count_oth=0
    for iSex in currSexes:
        if 'female' in iSex:
            count_fem+=1 
        elif 'male' in iSex:
            count_mal+=1
        elif 'NA' in iSex:
            count_oth+=1

    demogData.loc[iRow,'Sex_Female'] = count_fem
    demogData.loc[iRow,'Sex_Male'] = count_mal
    demogData.loc[iRow,'Sex_Other'] = count_oth

    #NDD
    currNDDs=NDDcol[rowIdx]
    count_aff=0
    count_not=0
    count_NA=0
    for iNDD in currNDDs:
        if iNDD == 'affected':
            count_aff+=1 
        elif iNDD == 'unaffected':
            count_not+=1
        elif 'NA' in iNDD or 'unknown' in iNDD:
            count_NA+=1

    demogData.loc[iRow,'NDD_Affected'] = count_aff
    demogData.loc[iRow,'NDD_NotAffected'] = count_not
    demogData.loc[iRow,'NDD_Unknown'] = count_NA

    
print(demogData)

      Count Age_Mean      Age_Range Sex_Female Sex_Male Sex_Other  \
P        23    19.81  [9.98, 44.42]         13        9         1   
M1       18    45.14  [34.98, 60.6]         18        0         0   
F1       12    46.95  [32.95, 60.8]          0       12         0   
S*       12    15.58   [9.89, 29.7]          5        7         0   
Total    65    31.05   [9.89, 60.8]         36       28         1   

      NDD_Affected NDD_NotAffected NDD_Unknown  
P               18               0           5  
M1               4               8           6  
F1               0               7           5  
S*               6               2           4  
Total           28              17          20  
